In [57]:
import sys
sys.version
import numpy as np

In [76]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.externals.joblib import dump
from sklearn.externals.joblib import load
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [68]:
filename = 'order_claim_ticket_v2_201612200055.csv'
dataframe = read_csv(filename)

In [69]:
dataframe.columns

Index(['﻿"order_id"', 'gender', 'title', 'given_name', 'middle_initial',
       'surname', 'street_address', 'city', 'state', 'zipcode', 'country',
       'country_full', 'email_address', 'username', 'password',
       'telephone_number', 'mothers_maiden', 'birthday', 'CC_type',
       'CC_number', 'cvv2', 'cc_expiration', 'national_id', 'color',
       'occupation', 'ref_domain', 'guid', 'latitude', 'longitude',
       'total_session_length', 'pageview_pre_purchase_visits', 'time_on_cart',
       'product_view_count', 'total_visit_clicks', 'fraud',
       'customer_credit_selection', 'payment_type', 'plan_type',
       'existing_ecom_customer', 'customer_id'],
      dtype='object')

In [70]:
dataframe = dataframe.loc[:,['total_session_length', 'pageview_pre_purchase_visits', 'time_on_cart',
       'product_view_count', 'total_visit_clicks', 'fraud',
       'customer_credit_selection', 'payment_type', 'plan_type']]

In [71]:
dataframe.head()

,total_session_length,pageview_pre_purchase_visits,time_on_cart,product_view_count,total_visit_clicks,fraud,customer_credit_selection,payment_type,plan_type
0,25.473872,70,10.170042,6,30,0,Good Credit,18 Month Lease,Small - 3GB
1,24.945612,71,9.295318,5,31,0,Good Credit,Full Price,Unlimited
2,25.840529,70,9.497712,4,30,0,Good Credit,Full Price,Medium - 6GB
3,26.004909,71,8.210692,6,29,0,Good Credit,24 Month Contract,Medium - 6GB
4,24.218376,70,10.313291,5,30,0,Good Credit,24 Month Contract,Unlimited


In [72]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 9 columns):
total_session_length            25000 non-null float64
pageview_pre_purchase_visits    25000 non-null int64
time_on_cart                    25000 non-null float64
product_view_count              25000 non-null int64
total_visit_clicks              25000 non-null int64
fraud                           25000 non-null int64
customer_credit_selection       25000 non-null object
payment_type                    25000 non-null object
plan_type                       25000 non-null object
dtypes: float64(2), int64(4), object(3)
memory usage: 1.7+ MB


In [73]:
array = dataframe.values
X = array[:,[0,1,2,3,4]]
Y = np.asarray(dataframe['fraud'], dtype="float")


In [95]:
num_trees = 100
max_features = 3
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=7)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [96]:
#save model to disk
filename = 'finalized_model.sav'
dump(model, open(filename, 'wb'))

In [97]:
# load the model from disk & Score in batch mode
loaded_model = load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

0.999151515152


In [109]:
#the same thing when scored 1 at a time
result = model.predict(X[59].reshape(1, -1))
print("the transaction is {}".format(result[0]))

the transaction is 1.0


In [105]:
from sklearn.externals.joblib import dump
from sklearn.externals.joblib import load

In [106]:
filename = 'forest_model.sav'
dump(model, filename)

['forest_model.sav']

In [113]:
# load the model from disk at a later time and score 1 at a time
loaded_model = load(filename)
result = loaded_model.predict(X[59].reshape(1, -1))
print("the transaction is {}".format(result[0]))

the transaction is 1.0


In [115]:
#Data that will be required to be passed to the streaming model
dataframe.iloc[:,[0,1,2,3,4]].head()

,total_session_length,pageview_pre_purchase_visits,time_on_cart,product_view_count,total_visit_clicks
0,25.473872,70,10.170042,6,30
1,24.945612,71,9.295318,5,31
2,25.840529,70,9.497712,4,30
3,26.004909,71,8.210692,6,29
4,24.218376,70,10.313291,5,30


In [117]:
dataframe.ix[59]

total_session_length                      17.3559
pageview_pre_purchase_visits                   59
time_on_cart                              1.76069
product_view_count                             10
total_visit_clicks                             21
fraud                                           1
customer_credit_selection             Good Credit
payment_type                    24 Month Contract
plan_type                               Unlimited
Name: 59, dtype: object